<a href="https://colab.research.google.com/github/michelebrooks/census-project/blob/main/CensusAnalysis_HousingStats_GA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Python Census Scraper for Housing Statistics in GA 
---
** A work in progress. Est. completion August 2022**


In [ ]:
!pip install google-cloud-secret-manager

In [ ]:
# packages for unzipping in memory
from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile
import os.path

# packages needed for census api calls
import requests
import pandas as pd
import geopandas as gpd
import shapefile
from google.cloud import secretmanager

# packages needed for transforming into nl json for BigQuery
from io import StringIO
import json
import geojson

In [ ]:
# Create a Client for Census API Call:
client = secretmanager.SecretManagerServiceClient()
secret_name = "census-api" 
project_id = 'tdl-freelance'

resource_name = f"projects/{project_id}/secrets/{secret_name}/versions/latest" 

response = client.access_secret_version(request={"name": resource_name})
secret_string = response.payload.data.decode('UTF-8')

In [ ]:
# Set variables to pull from Census API
acs_vars_dict = {'DP02_0059E': 'edu_pop25plus', # Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 years and over
                 'DP02_0067E': 'edu_pop25plus_hsPlus', #Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate or higher
                 'DP02_0068E': 'edu_pop25plus_baPlus', #Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelors degree or higher
                 'DP02_0066E': 'edu_pop25plus_maPlus', #Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Graduate or professional degree
                 'DP02_0152E': 'total_households', #Estimate!!COMPUTERS AND INTERNET USE!!Total households
                 'DP02_0153E': 'tech_hh_computer', #Estimate!!COMPUTERS AND INTERNET USE!!Total households!!With a computer
                 'DP02_0154E': 'tech_hh_internetsub', #Estimate!!COMPUTERS AND INTERNET USE!!Total households!!With a broadband Internet subscription
                 'DP03_0001E': 'emp_pop16plus', #Estimate!!EMPLOYMENT STATUS!!Population 16 years and over
                 'DP03_0003E': 'emp_pop16plus_lf_civ', #Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Civilian labor force
                 'DP03_0004E': 'emp_pop16plus_lf_civ_employed', #Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Civilian labor force!!Employed
                 'DP03_0005E': 'emp_pop16plus_lf_civ_employed_unemployed', #Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Civilian labor force!!Unemployed
                 'DP03_0006E': 'emp_pop16plus_lf_army', #Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Armed Forces
                 'DP03_0007E': 'emp_pop16plus_notlf', #
                 'DP03_0062E': 'income_hh_median', #Estimate!!INCOME AND BENEFITS (IN 2020 INFLATION-ADJUSTED DOLLARS)!!Total households!!Median household income (dollars)
                 'DP03_0063E': 'income_hh_mean', #Estimate!!INCOME AND BENEFITS (IN 2020 INFLATION-ADJUSTED DOLLARS)!!Total households!!Mean household income (dollars)
                 'DP03_0119E': 'income_hh_poverty', #Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!All families
                 'DP04_0001E': 'total_housing_units', #Estimate!!HOUSING OCCUPANCY!!Total housing units
                 'DP04_0002E': 'occ_hu_occupied', #Estimate!!HOUSING OCCUPANCY!!Total housing units!!Occupied housing units
                 'DP04_0003E': 'occ_hu_vacant', #Estimate!!HOUSING OCCUPANCY!!Total housing units!!Vacant housing units
                 'DP04_0004E': 'occ_hu_homeowner_vacancy_rate', #Estimate!!HOUSING OCCUPANCY!!Total housing units!!Homeowner vacancy rate
                 'DP04_0007E': 'units_hu_1unit_detached', #Estimate!!UNITS IN STRUCTURE!!Total housing units!!1-unit, detached
                 'DP04_0008E': 'units_hu_1unit_attached', #Estimate!!UNITS IN STRUCTURE!!Total housing units!!1-unit, attached
                 'DP04_0009E': 'units_hu_2units', # Estimate!!UNITS IN STRUCTURE!!Total housing units!!2 units
                 'DP04_0010E': 'units_hu_3to4units', # Estimate!!UNITS IN STRUCTURE!!Total housing units!!3 or 4 units
                 'DP04_0011E': 'units_hu_5to9units', # Estimate!!UNITS IN STRUCTURE!!Total housing units!!5 to 9 units
                 'DP04_0012E': 'units_hu_10to19units', # Estimate!!UNITS IN STRUCTURE!!Total housing units!!10 to 19 units
                 'DP04_0013E': 'units_hu_20plusunits', # Estimate!!UNITS IN STRUCTURE!!Total housing units!!20 or more units
                 'DP04_0014E': 'units_hu_mobilehome', # Estimate!!UNITS IN STRUCTURE!!Total housing units!!Mobile home
                 'DP04_0080E': 'value_owner_occupied_units', # Estimate!!VALUE!!Owner-occupied units
                 'DP04_0081E': 'value_oo_less50k', # Estimate!!VALUE!!Owner-occupied units!!Less than $50,000
                 'DP04_0082E': 'value_oo_50k99k', # Estimate!!VALUE!!Owner-occupied units!!$50,000 to $99,999
                 'DP04_0083E': 'value_oo_100k149k', # Estimate!!VALUE!!Owner-occupied units!!$100,000 to $149,999
                 'DP04_0084E': 'value_oo_150k199k', # Estimate!!VALUE!!Owner-occupied units!!$150,000 to $199,999
                 'DP04_0085E': 'value_oo_200k299k', # Estimate!!VALUE!!Owner-occupied units!!$200,000 to $299,999
                 'DP04_0086E': 'value_oo_300k499k', # Estimate!!VALUE!!Owner-occupied units!!$300,000 to $499,999
                 'DP04_0087E': 'value_oo_500k999k', # Estimate!!VALUE!!Owner-occupied units!!$500,000 to $999,999
                 'DP04_0088E': 'value_oo_1mplus', # Estimate!!VALUE!!Owner-occupied units!!$1,000,000 or more
                 'DP04_0089E': 'value_oo_median', # Estimate!!VALUE!!Owner-occupied units!!Median (dollars)
                 'STATE': 'state',
                 'COUNTY': 'county',
                 'TRACT': 'tract'
                }
acs_vars = list(acs_vars_dict.keys())
acs_names = list(acs_vars_dict.values())

print(acs_vars)
print(acs_names)

In [ ]:
# generating request for Census API call
HOST = "https://api.census.gov/data"
year = "2020"
dataset = "acs/acs5/profile"
base_url = "/".join([HOST, year, dataset])
predicates = {}
get_vars = str(','.join(acs_vars[:-3]))
in_vars = ["state:13", "county:067"]
predicates["get"] = get_vars
predicates["for"] = "tract:*"
predicates["in"] = "&in=".join(in_vars)
#predicates["key"] = api_key
r = requests.get(base_url, params=predicates)

# putting api call results into data fram with formatted column names
attr_df = pd.DataFrame(columns=acs_names, data=r.json()[1:])

In [ ]:
# unzip GA Census Tracts shapefile zip in memory and save all files gis_layers/tract
if not os.path.exists('gis_layers/tract/tl_2021_13_tract.shp'):
    zipurl = 'https://www2.census.gov/geo/tiger/TIGER2021/TRACT/tl_2021_13_tract.zip'
    with urlopen(zipurl) as tract_zipresp:
        with ZipFile(BytesIO(tract_zipresp.read())) as zfile:
            zfile.extractall('gis_layers/tract')

In [ ]:
# Load tract geojson and filter to Cobb County
df = gpd.read_file('gis_layers/tract/ga_tract.geojson')
geo_df=df[df["COUNTYFP"]=='067']